# Lec 10B: Multi-Agentic RAG using Tools

###Create Agents to Research (from a pdf) and Write an Email

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
#Use your directory here...
%cd /content/drive/My Drive/Courses/MGT802-LLM/MGT802-LLM-2025/ClassNotebooks

Mounted at /content/drive
/content/drive/My Drive/Courses/MGT802-LLM/MGT802-LLM-2025/ClassNotebooks


In [2]:
import os
from google.colab import userdata
os.environ['OPENAI_API_KEY']   = userdata.get('my_openaikey')
#By including model name here, all agents use this model
#We can use different models for different agents by specifying it within agents
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o-mini'

In [ ]:
%%capture
!pip install --upgrade crewai crewai_tools  qdrant-client==1.15.1 langchain_community

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
from crewai import Agent, Task, Crew, Process
from crewai_tools import PDFSearchTool
#from langchain_openai import ChatOpenAI

###Agentic RAG
1. Research Agent answers customer questions from a PDF document (RAG)-A home inspection report
2. Writer Agent drafts an email automatically --sending an email to a contractor for a quote, building on the question

In [ ]:
#Load the pdf and set it up for search (here we load from the URL directy)
pdf_search_tool = PDFSearchTool("Home Inspection Report.pdf")

# --- Agents ---
research_agent = Agent(
    role="Research Agent",
    goal="Search through the PDF to find relevant answers",
    allow_delegation=False,
    verbose=True,
    backstory=(
        """
        The research agent is adept at searching and
        extracting data from documents, ensuring accurate and prompt responses.
        """
    ),
    tools=[pdf_search_tool],

)

professional_writer_agent = Agent(
    role="Professional Writer",
    goal="Write professional emails based on the research agent's findings",
    allow_delegation=False,
    verbose=True,
    backstory=(
        """
        The professional writer agent has excellent writing skills and is able to craft
        clear and concise emails based on the provided information.
        """
    ),
    tools=[],
)


# --- Tasks ---
answer_customer_question_task = Task(
    description=(
        """
        Answer the customer's questions based on the home inspection PDF.
        The research agent will search through the PDF to find the relevant answers.
        Your final answer MUST be clear and accurate, based on the content of the home
        inspection PDF.

        Here is the customer's question:
        {customer_question}
        """
    ),
    expected_output="""
        Provide clear and accurate answers to the customer's questions based on
        the content of the home inspection PDF.
        """,
    tools=[pdf_search_tool],
    agent=research_agent,
)

write_email_task = Task(
    description=(
        """
        - Write a professional email to a contractor based
            on the research agent's findings.
        - The email should clearly state the issues found in the specified section
            of the report and request a quote or action plan for fixing these issues.
        - Ensure the email is signed with the following details:

            Best regards,

            Richard Archer,
            802 Realtors
        """
    ),
    expected_output="""
        Write a clear and concise email that can be sent to a contractor to address the
        issues found in the home inspection report.
        """,
    tools=[],
    agent=professional_writer_agent,
)

# --- Crew ---
crew = Crew(
    agents=[research_agent, professional_writer_agent],
    tasks=[answer_customer_question_task, write_email_task],
    process=Process.sequential,
)

customer_question = input(
    "Which section of the report would you like to generate a work order for?\n"
)
# You can open the pdf to see the sections, but even if you ask related questions, the Agent may answer
result = crew.kickoff(inputs={"customer_question": customer_question})

# Display the result as Markdown
from IPython.display import Markdown
Markdown(result.raw)